In [1]:
import pandas as pd

Gives us more space to use horizontally:

In [2]:
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 75% !important; }</style>"))

In [3]:
from glob import glob

In [4]:
allVideos = glob("Videos/*/*.mp4")

In [5]:
allVideos

['Videos\\P01\\P01-20240202-110250.mp4',
 'Videos\\P01\\P01-20240202-161354.mp4',
 'Videos\\P01\\P01-20240202-161948.mp4',
 'Videos\\P01\\P01-20240202-171220.mp4',
 'Videos\\P01\\P01-20240202-175627.mp4',
 'Videos\\P01\\P01-20240202-195538.mp4',
 'Videos\\P01\\P01-20240203-093333.mp4',
 'Videos\\P01\\P01-20240203-121517.mp4',
 'Videos\\P01\\P01-20240203-123350.mp4',
 'Videos\\P01\\P01-20240203-130505.mp4',
 'Videos\\P01\\P01-20240203-132119.mp4',
 'Videos\\P01\\P01-20240203-135502.mp4',
 'Videos\\P01\\P01-20240203-150506.mp4',
 'Videos\\P01\\P01-20240203-152323.mp4',
 'Videos\\P01\\P01-20240203-152956.mp4',
 'Videos\\P01\\P01-20240203-161757.mp4',
 'Videos\\P01\\P01-20240203-184045.mp4',
 'Videos\\P01\\P01-20240203-184214.mp4',
 'Videos\\P01\\P01-20240204-095114.mp4',
 'Videos\\P01\\P01-20240204-120411.mp4',
 'Videos\\P01\\P01-20240204-121042.mp4',
 'Videos\\P01\\P01-20240204-124504.mp4',
 'Videos\\P01\\P01-20240204-130448.mp4',
 'Videos\\P01\\P01-20240204-142301.mp4',
 'Videos\\P01\\P

In [6]:
new_annot_filename = "HD_EPIC_Sounds_annot.csv"
try: 
    new_annot_df = pd.read_csv(new_annot_filename) 
except:
    new_annot_df = pd.read_csv("HD_EPIC_Sounds.csv")

tap_class = "tap water"
tap_class_id = 44   # 45th class added. Will mostly be very similar to water and pour, but only active for actual tap water sounds!


In [7]:
import pathlib


In [8]:
pathlib.Path(allVideos[0]).name[:-4]

'P01-20240202-110250'

In [9]:
annot_csv = pd.read_csv("HD_EPIC_Sounds.csv")

all_rec_ids = annot_csv.video_id.unique()

water_df = annot_csv[(annot_csv["class"] == "water")|(annot_csv["class"]=="pour")]

water_df = water_df[(water_df.stop_sample - water_df.start_sample) / 48000 > 3]

In [10]:
allVideos

['Videos\\P01\\P01-20240202-110250.mp4',
 'Videos\\P01\\P01-20240202-161354.mp4',
 'Videos\\P01\\P01-20240202-161948.mp4',
 'Videos\\P01\\P01-20240202-171220.mp4',
 'Videos\\P01\\P01-20240202-175627.mp4',
 'Videos\\P01\\P01-20240202-195538.mp4',
 'Videos\\P01\\P01-20240203-093333.mp4',
 'Videos\\P01\\P01-20240203-121517.mp4',
 'Videos\\P01\\P01-20240203-123350.mp4',
 'Videos\\P01\\P01-20240203-130505.mp4',
 'Videos\\P01\\P01-20240203-132119.mp4',
 'Videos\\P01\\P01-20240203-135502.mp4',
 'Videos\\P01\\P01-20240203-150506.mp4',
 'Videos\\P01\\P01-20240203-152323.mp4',
 'Videos\\P01\\P01-20240203-152956.mp4',
 'Videos\\P01\\P01-20240203-161757.mp4',
 'Videos\\P01\\P01-20240203-184045.mp4',
 'Videos\\P01\\P01-20240203-184214.mp4',
 'Videos\\P01\\P01-20240204-095114.mp4',
 'Videos\\P01\\P01-20240204-120411.mp4',
 'Videos\\P01\\P01-20240204-121042.mp4',
 'Videos\\P01\\P01-20240204-124504.mp4',
 'Videos\\P01\\P01-20240204-130448.mp4',
 'Videos\\P01\\P01-20240204-142301.mp4',
 'Videos\\P01\\P

In [11]:
len(allVideos)

156

In [12]:
from tqdm.auto import tqdm

In [13]:
import ipywidgets as widgets
import plotly.graph_objects as go
import pandas as pd
from IPython.display import display, HTML, Javascript, Video

# Ensure the required packages are installed
try:
    import plotly
    import ipywidgets
    import pandas
except ImportError:
    !pip install plotly ipywidgets pandas
    import plotly
    import ipywidgets
    import pandas


def annotate_single_video(video_path):
    rec_id = pathlib.Path(video_path).name[:-4]
    # Create video display using HTML

    video_wid = widgets.Output()

    with video_wid:
        display(HTML(f"""
            <video id="video_player" width="400" controls>
                <source src="{video_path}" type="video/mp4">
                Your browser does not support the video tag.
            </video>
        """))
        
    video_df = water_df[water_df.video_id == rec_id].copy()
    video_df["Start"] = video_df.start_sample / 48000
    video_df["End"] = video_df.stop_sample / 48000

    if len(video_df) == 0:
        print("Video does not contain any water or pour annotations!")
        return
    
    
    slider = widgets.FloatSlider(
        min=0, max=max(video_df["End"]) + 5, step=0.005, description="Time:", continuous_update=True)
    
    out = widgets.Output()
    
    
    def update_video_float(fl):
        update_video({"new": fl})
    # Function to update video playback based on slider
    def update_video(change):
        js = f"document.querySelector('video').currentTime = {change['new']};"
        with out:        
            display(Javascript(js))
        out.clear_output()
    
    slider.observe(update_video, names="value")
    
    # Create an interactive timeline using Plotly
    fig = go.FigureWidget()


    
    def update_timeline():
        fig.data = []
        for i, row in video_df.iterrows():
            fig.add_trace(go.Scatter(
                x=[row["Start"], row["End"]],
                y=[row["class"], row["class"]],
                mode="lines+markers",
                line=dict(width=6),
                marker=dict(size=12),
                name=row["class"],
                customdata=[i],
            ))
    
    update_timeline()
    
    # Combine widgets into a single output
    ui = widgets.VBox([widgets.HBox([video_wid, fig, out]), slider])
    
    
    display(ui)

    video_df = water_df[water_df.video_id == rec_id].copy()
    video_df["Start"] = video_df.start_sample / 48000
    video_df["End"] = video_df.stop_sample / 48000

    print(f"In this video, {rec_id}, there are", len(video_df), "annotations to check!")
    for index, line in tqdm(video_df.iterrows(), total=len(video_df)):
        fig.update_layout(xaxis=dict(range=[line.Start-15,line.End+15]))
        update_video({"new": line.Start})
        slider.max = line.End + 60
        slider.min = line.Start - 60
        slider.value = line.Start

        while True:
            new_index = len(new_annot_df)
            r = input(f"change annotation ({int(line.End - line.Start)}s)? {line.Start},{line.End}:")
            if r == "":
                # keep annotation the same
                
                new_annot_df.loc[new_index] = line
                new_annot_df.loc[new_index, "class"] = tap_class
                new_annot_df.loc[new_index, "class_id"] = tap_class_id
                print("keeping annotation")
                break
            elif r == "d":
                print("dropped")
                break
            elif type(eval(r)) is not tuple:
                print(r, type(r))
                r = eval(input("wrong type entered, try again!:"))
            else:
                # r is a tuple, of start and end seconds.
                r = eval(r)
                start = r[0]
                stop = r[1]
                add_additional_annotation(start, stop, line.participant_id, rec_id)
                break
    ui.close()

In [14]:
def add_additional_annotation(start, stop, part_id, video_id):
    start_sample = int(start * 48000)
    stop_sample = int(stop * 48000)
    start_timestamp = get_timestamp(start)
    stop_timestamp = get_timestamp(stop)
    new_annot_df.loc[len(new_annot_df)] =  [part_id, video_id, start_timestamp, stop_timestamp, start_sample, stop_sample, tap_class, tap_class_id]

def get_timestamp(seconds):
    # returns a string, timestamp with HH:MM:SS.mmm.
    hours = seconds // 3600
    left = seconds - hours * 3600
    minutes = left // 60
    left = left - minutes * 60
    
    hours = f"{int(hours):d}"
    if len(hours) == 1:
        hours = "0" + hours

    minutes = str(int(minutes))
    if len(minutes) == 1:
        minutes = "0" + minutes
    #print(hours, minutes, left)

    seconds = f"{left:.3f}"
    if len(seconds) == 5:
        seconds = "0" + seconds
    
    return f"{hours}:{minutes}:{seconds}"

In [15]:
water_df.class_id.unique()

array([ 5, 19], dtype=int64)

In [16]:
new_annot_df[new_annot_df.class_id==tap_class_id]

,participant_id,video_id,start_timestamp,stop_timestamp,start_sample,stop_sample,class,class_id
50967,P08,P08-20240614-085000,00:13:54.107,00:13:58.082,40037136,40227936,tap water,44
50968,P08,P08-20240614-085000,00:14:57.119,00:15:04.732,43061712,43427136,tap water,44
50969,P08,P08-20240614-085000,00:15:13.957,00:15:16.900,43869936,44011200,tap water,44
50970,P08,P08-20240614-085000,00:15:19.657,00:15:31.594,44143536,44716512,tap water,44
50971,P08,P08-20240614-085000,00:15:35.950,00:15:41.300,44925600,45182400,tap water,44
...,...,...,...,...,...,...,...,...
51524,P08,P08-20240617-184909,00:26:13.951,00:26:18.312,75549648,75758976,tap water,44
51525,P08,P08-20240617-184909,00:28:49.685,00:28:58.500,83024880,83448000,tap water,44
51526,P08,P08-20240617-184909,00:29:05.170,00:29:14.125,83768160,84198000,tap water,44
51527,P08,P08-20240617-184909,00:29:40.811,00:29:51.950,85478928,86013600,tap water,44


In [17]:
def get_done_count(done):
    done_ids = [v[11:-4] for v in done]
    count = 0
    for video_id, line in water_df.groupby("video_id").count().iterrows():
        if video_id in done_ids:
            count += line.participant_id
    return count

In [18]:
def remove_from_done(video_file):
    with shelve.open("done_vids") as shf:
        done = shf.get("done", [])
        if video_file in done:
            done.remove(video_file)
            shf["done"]= done

In [19]:
new_annot_df

,participant_id,video_id,start_timestamp,stop_timestamp,start_sample,stop_sample,class,class_id
0,P01,P01-20240202-110250,00:00:00.476,00:00:02.520,22848,120960,rustle,4
1,P01,P01-20240202-110250,00:00:03.767,00:00:07.888,180816,378624,footstep,7
2,P01,P01-20240202-110250,00:00:06.462,00:00:06.682,310176,320736,footstep,7
3,P01,P01-20240202-110250,00:00:07.765,00:00:08.667,372720,416016,open / close,3
4,P01,P01-20240202-110250,00:00:09.567,00:00:10.342,459216,496416,ceramic-only collision,14
...,...,...,...,...,...,...,...,...
51524,P08,P08-20240617-184909,00:26:13.951,00:26:18.312,75549648,75758976,tap water,44
51525,P08,P08-20240617-184909,00:28:49.685,00:28:58.500,83024880,83448000,tap water,44
51526,P08,P08-20240617-184909,00:29:05.170,00:29:14.125,83768160,84198000,tap water,44
51527,P08,P08-20240617-184909,00:29:40.811,00:29:51.950,85478928,86013600,tap water,44


In [37]:
import shelve
with shelve.open("done_vids") as shf:
    done = shf.get("done", [])

try: 
    new_annot_df = pd.read_csv(new_annot_filename) 
except:
    new_annot_df = pd.read_csv("HD_EPIC_Sounds.csv")

done_vid_list = new_annot_df[new_annot_df.class_id==tap_class_id].video_id.unique()

rest_videos = [vid_filename for vid_filename in allVideos if vid_filename not in done]

for i, video in enumerate(rest_videos):
    new_annot_df = new_annot_df.drop_duplicates()
    new_annot_df.to_csv(new_annot_filename, index=False)
    print(len(rest_videos) - i, "videos left,", get_done_count(done), "of 1098 annotations checked")
    annotate_single_video(video)
    print("finished video", video)
    new_annot_df = new_annot_df.drop_duplicates()
    new_annot_df.to_csv(new_annot_filename, index=False)
    print(len(new_annot_df[new_annot_df.class_id==tap_class_id]), "annotations added so far")
    with shelve.open("done_vids") as shf:
        done = shf.get("done", [])
        done.append(video)
        shf["done"] = done
if len(rest_videos) == 0:
    print("no videos left, everything is done!")

no videos left, everything is done!


In [31]:
#video_id = "P09-20240622-194642"
#add_additional_annotation(260.75, 284.2, video_id[:3], video_id)


In [ ]:
widgets.HTML("""<video width="400" height="400" controls><source src="Videos\\P01\\P01-20240202-195538.mp4" type="video/mp4"></video>""")

In [25]:
# wrong: P09-20240621-093545 1990.447,2017.215: 1990.447,2015.9, should be 1995.9

In [39]:
tmp_df = new_annot_df.copy()

Found errors:

- Wishking an egg labeled as "water"
- Pour annotated as water instead of pour
- Stiring a pot annotated as water, e.g. P08-20240618-171546 but happens multiple times, even if there is no "watery" sound
- overlapping water labels (one for general water, one for tap water, e.g. P01-20240202-175627, 87s-93s is contained), "nested annotations"
- Moving food into a container labeled as water
- water in far background, e.g. P01-20240203-152323 3-14s
- single water drops in background labeled as water, but maybe not consistently (check: maybe cause i dropped the labels, eg. P02 first video)

- P01 water "in drain / pipes" annotated, not for P02 194141
- P02: P02-20240209-194141 - water annotations inconsistent while doing dishes
- Sizzeling pan in background annotated as water P02-20240210-113925, 490.982,530.482 or P03-20240217-192543, 985.078,990.185
- Duplicated annotations: e.g. P03-20240217-210958 - 19:20-19:55
- Plastic Bag handling sounds annotated as pour, P04-20240414-162750: 1:19 onward
- Steam cooker letting out steam annotated as water P04-20240414-165333 - technically correct but different form and sound
- plating (non-liquid) food from a pan labeled as water P04-20240414-175337
- In P06-20240510-100047, there is a "pour" annotation for someone frothing milk
- 
  

Found explanations for problems:

- P02:
  - very silent tap, sometimes running in the background far away
  - sometimes only a drizzle, almost inaudible, e.g. P02-20240211-17513 190-226
- P03:
  - P03-20240216-185832, 28:42 : extractor hood very loud in the background, similar noise to water, water barely hearable
- P04:
  - P04-20240414-065311: Tap technically on, but only as a very slight drizzle - almost inaudible, no "noise"
- P06:
  - P06-20240510-104642: Loud noise in background (dunstabzug)
  - 

In [1]:
new_annot_df

NameError: name 'new_annot_df' is not defined

In [ ]:
new_label_df

In [ ]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

In [ ]:
tmp_df[tmp_df.class_id==tap_class_id]

In [ ]:
tmp_df.sort_values(by=["participant_id","video_id","start_sample"]).reset_index(drop=True)

In [ ]:
from IPython.display import HTML
import ipywidgets as widgets

# Create an Output widget to hold the HTML video player
video_output = widgets.Output()

# Embed the video using IPython HTML inside the output widget
with video_output:
    display(HTML("""
    <video width="400" height="400" controls>
        <source src="Videos/P01/P01-20240202-195538.mp4" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """))


In [ ]:
import matplotlib.pyplot as plt

# Assume df is your DataFrame
# Set the participant you want to analyze
participant_id = "P01"
sample_distance = 10* 48000  # Adjust this as needed

# Filter for the selected participant
df_participant = tmp_df[(tmp_df["participant_id"] == participant_id) & ((tmp_df.stop_sample-tmp_df.start_sample)/ 48000 >= 3)]

video_ids = df_participant["video_id"].unique()

# Create subplots — one per video
fig, axes = plt.subplots(len(video_ids), 1, figsize=(12, 4 * len(video_ids)), sharex=False)

# If only one subplot, wrap in list for consistency
if len(video_ids) == 1:
    axes = [axes]

for ax, video_id in zip(axes, video_ids):
    df_video = df_participant[df_participant["video_id"] == video_id]
    water_labels = df_video[df_video["class"] == "water"]
    tap_water_labels = df_video[df_video["class"] == "tap water"]

    n_water = len(water_labels)
    n_tap = len(tap_water_labels)

    # Plot water and nearby tap water
    for idx, row in water_labels.iterrows():
        ax.axvspan(row["start_sample"], row["stop_sample"], color='blue', alpha=0.5, label="water" if idx == water_labels.index[0] else "")
        
        nearby_tap_water = tap_water_labels[
            ((tap_water_labels["start_sample"] >= row["start_sample"] - sample_distance) &
             (tap_water_labels["start_sample"] <= row["stop_sample"] + sample_distance))
        ]
        
        for jdx, tap_row in nearby_tap_water.iterrows():
            ax.axvspan(tap_row["start_sample"], tap_row["stop_sample"], color='green', alpha=0.5, label="tap water" if jdx == nearby_tap_water.index[0] else "")

    ax.set_title(f"Video ID: {video_id}, water:{n_water}, tap:{n_tap}")
    #ax.set_xlabel("Sample Index")
    ax.set_ylabel("Activity")
    ax.legend(loc="upper right")

plt.show()